In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-dq3dmbe_
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-dq3dmbe_
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4294 sha256=cb35b3c88ec2207ed7309794d371131d0dfbdaa187f441e34fdf522a0330b4c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-_xq_dj39/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [57]:
%%cuda --name MIS.cu

#include "/content/drive/MyDrive/graphcoloring/graph/coloring.h"
#include "/content/drive/MyDrive/graphcoloring/graph/graph.h"
#include "/content/drive/MyDrive/graphcoloring/graph/graph_d.h"
#include "/content/drive/MyDrive/graphcoloring/utils/common.h"

#define THREADxBLOCK 128

using namespace std;

__device__ bool result = true;

__global__ void colorer(Coloring * col, GraphStruct *str){
	int n = str->nodeSize;

	for (int i = threadIdx.x+blockIdx.x*blockDim.x; i < n; i += blockDim.x*gridDim.x) {
		bool flag = true; // vera sse il nodo ha peso locale massimo

		// ignora i nodi già colorati
		if ((col->coloring[i] != -1)) continue;

		int iWeight = str->weights[i];

		// guarda i pesi del vicinato
		uint offset = str->cumDegs[i];
		uint deg = str->cumDegs[i + 1] - str->cumDegs[i];

		for (uint j = 0; j < deg; j++) {
			uint neighID = str->neighs[offset + j];
			// ignora i vicini già colorati (e te stesso)
			int jColor = col->coloring[neighID];
			if (((jColor != -1) && (jColor != col->numOfColors)) || jColor == -2 || (i == neighID)) continue;
			int jWeight = str->weights[neighID];
			if (iWeight <= jWeight) flag = false;
		}
		// colora solo se sei il nodo di peso massimo
		if (flag){
			col->coloring[i] = col->numOfColors;
			for (uint j = 0; j < deg; j++) {
				uint neighID = str->neighs[offset + j];
				if(col->coloring[neighID] == -1) col->coloring[neighID] = -2;
			}
		}
	}
}

void FYshuffle(int * weights, uint n){
	for(int i = 0; i < n; i++){
		int swapIdx = (rand() % (n - i)) + i;
		int tmp = weights[i];
		weights[i] = weights[swapIdx];
		weights[swapIdx] = tmp;
	}
}

Coloring* graphColoring(GraphStruct *str){
	int n = str->nodeSize;
	Coloring* col;
	CHECK(cudaMallocManaged(&col, sizeof(Coloring)));
	col->uncoloredNodes = true;

	CHECK(cudaMallocManaged(&(col->coloring), n * sizeof(int)));
	memset(col->coloring, -1 ,n * sizeof(int));
	// allocate space on the GPU for the random states

	dim3 threads ( THREADxBLOCK);
	dim3 blocks ((str->nodeSize + threads.x - 1) / threads.x, 1, 1 );

	for (int i = 0; i < n; i++){
				str->weights[i] = i;
		}

	FYshuffle(str->weights, n);

	for (int i = 0; i < n; i++){
				printf("%d ", str->weights[i]);
		}
	printf("\n");

	//print_d <<< 1, 1 >>> (str, true);

	bool flag=true;
	while(col->numOfColors<n){
		while(flag){
			colorer<<<blocks, threads>>>(col, str);
			cudaDeviceSynchronize();
			flag=false;
			for(int i=0; i<n; i++){
				if(col->coloring[i]==-1){
						flag=true;
				}
			}
		}
		flag=false;
		for(int i=0; i<n; i++){
			if(col->coloring[i]==-2){
					flag=true;
					col->coloring[i]=-1;
			}
		}
		if(!flag) break;
		col->numOfColors++;
	}

	return col;
}

'File written in /content/src/MIS.cu'

In [63]:
%%cuda --name test_MIS.cu

#include "/content/drive/MyDrive/graphcoloring/graph/coloring.h"

using namespace std;

int main(void) {
	unsigned int n = 100;		 // number of nodes for random graphs
	float prob = 0.5;				    // density (percentage) for random graphs
	std::default_random_engine eng{0};  // fixed seed

  srand(time(0));
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

	// new graph with n nodes
	Graph graph(n,1);

	// generate a random graph
	graph.randGraph(prob,eng);

	// get the graph struct
	GraphStruct *str = graph.getStruct();

  cudaEventRecord(start);

	Coloring* col = graphColoring(str);
	cudaDeviceSynchronize();

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);

  //Stampo in millisecondi quanto tempo ci ha messo a colorare il grafo.
  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  printf("%f ms\n", milliseconds);

	printf("Colorazione: ");
	for(int i = 0; i < n; i++){
			printf("%d ", col->coloring[i]);
	}

	printf("\nColore massimo: %d", col->numOfColors+1);

	return EXIT_SUCCESS;
}

'File written in /content/src/test_MIS.cu'

In [65]:
!nvcc -dc -D__CDPRT_SUPPRESS_SYNC_DEPRECATION_WARNING src/test_MIS.cu /content/src/MIS.cu /content/drive/MyDrive/graphcoloring/graph/graph.cpp /content/drive/MyDrive/graphcoloring/graph/graph_d.cu
!nvcc test_MIS.o MIS.o graph.o graph_d.o -o test_MIS
!./test_MIS

62 17 3 57 94 33 41 21 39 95 63 92 45 77 80 59 79 91 56 6 46 29 96 32 24 65 98 15 71 89 90 75 52 0 70 11 88 85 51 48 28 18 72 40 83 16 66 74 68 14 31 84 61 22 50 58 10 47 93 60 97 20 37 25 78 9 64 34 8 5 19 76 86 73 38 43 35 49 23 81 42 99 13 7 55 69 53 30 27 36 26 67 87 44 82 2 4 1 54 12 
9.547424 ms
Colorazione: 6 5 20 0 0 14 10 17 7 3 8 3 13 7 1 12 3 4 9 18 9 16 2 8 14 0 1 12 0 2 0 6 11 5 0 10 3 2 12 13 2 13 1 2 6 16 8 7 1 2 14 5 10 17 5 10 12 9 1 11 2 8 11 7 7 5 9 12 17 18 14 6 1 8 4 11 6 4 13 4 4 0 15 19 1 5 6 15 9 1 11 7 4 10 5 21 19 12 5 18 
Colore massimo: 22

In [66]:
!nvprof ./test_MIS

==26133== NVPROF is profiling process 26133, command: ./test_MIS
62 17 3 57 94 33 41 21 39 95 63 92 45 77 80 59 79 91 56 6 46 29 96 32 24 65 98 15 71 89 90 75 52 0 70 11 88 85 51 48 28 18 72 40 83 16 66 74 68 14 31 84 61 22 50 58 10 47 93 60 97 20 37 25 78 9 64 34 8 5 19 76 86 73 38 43 35 49 23 81 42 99 13 7 55 69 53 30 27 36 26 67 87 44 82 2 4 1 54 12 
11.234816 ms
Colorazione: 6 5 20 0 0 14 10 17 7 3 8 3 13 7 1 12 3 4 9 18 9 16 2 8 14 0 1 12 0 2 0 6 11 5 0 10 3 2 12 13 2 13 1 2 6 16 8 7 1 2 14 5 10 17 5 10 12 9 1 11 2 8 11 7 7 5 9 12 17 18 14 6 1 8 4 11 6 4 13 4 4 0 15 19 1 5 6 15 9 1 11 7 4 10 5 21 19 12 5 18 
Colore massimo: 22==26133== Profiling application: ./test_MIS
==26133== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   94.15%  6.1638ms        56  110.07us  24.192us  599.45us  colorer(Coloring*, GraphStruct*)
                    5.85%  383.13us         1  383.13us  383.13us  383.13us  [CUDA memset]
      